<a href="https://colab.research.google.com/github/shinhyunji36/codestates-ai-project/blob/main/AI_03_%EC%8B%A0%ED%98%84%EC%A7%80_Section1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Section1 Project**
- AI_03 신현지
- 준비 기간 : 2021.05.28 ~ 2021. 06.02

- 프로젝트 목표 : **다음 분기에 어떤 게임을 설계해야하는가**
- 조사 방법 : **데이터 기반 의사결정** 과정과 그 결과 제시
- 발표 대상 : **(데이터 분석에 관한 배경지식이 없는) 비데이터 직군**


#**Section1 keyword**
- Sprint 1 : EDA, Feature Engineering, Data Manipulation, Data Visualization
- Sprint 2 : Hypothesis Test, CI, Bayesian
- Sprint 3 : DR, PCA, Cluster

# **Data Description**
- data type : game sales
- Features :

  1) Name : 게임의 이름

  2) Platform : 게임이 지원되는 플랫폼의 이름

  3) Year : 게임이 출시된 연도

  4) Genre : 게임의 장르

  5) Publisher : 게임을 제작한 회사

  6) NA_Sales : 북미지역에서의 출고량

  7) EU_Sales : 유럽지역에서의 출고량

  8) JP_Sales : 일본지역에서의 출고량

  9) Other_Sales : 기타지역에서의 출고량

  * *Sales feature 내 단위 : k = 1000(천), m = 1000000(백만)*

#**공통 사항**
1. 지역에 따라서 선호하는 게임 장르가 다를까
2. 연도별 게임의 트렌드가 있을까
3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스

# **1. EDA**
###**1-1. 데이터 준비 및 전처리**

In [2]:
# 로컬 데이터 불러오기
from google.colab import files
uploaded = files.upload()

Saving vgames2.csv to vgames2.csv


In [3]:
# 기본 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [143]:
# csv파일 불러오기
df = pd.read_csv("vgames2.csv",index_col=1).iloc[:,1:]
df.head()

,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Name,,,,,,,,
Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [11]:
# 데이터 타입 확인 : dtypes, 직접 데이터 열어 확인(엑셀 필터로 간추린 확인)
df.dtypes

Platform        object
Year           float64
Genre           object
Publisher       object
NA_Sales        object
EU_Sales        object
JP_Sales        object
Other_Sales     object
dtype: object

In [ ]:
# feature에 맞지 않는 값 들어있나 value_counts로 내부 값들과 그 값의 개수 확인
df['Platform'].value_counts(normalize=False, sort=True, ascending=False,bins=None,dropna=False)
df['Year'].value_counts(normalize=False, sort=True, ascending=True,bins=None,dropna=False).head(40)
df['Other_Sales'].value_counts(normalize=False, sort=True, ascending=True,bins=None,dropna=False).head(30)
df['EU_Sales'].value_counts(normalize=False, sort=True, ascending=True,bins=None,dropna=False).head(30)
df['NA_Sales'].value_counts(normalize=False, sort=True, ascending=True,bins=None,dropna=False).head(30)
df['JP_Sales'].value_counts(normalize=False, sort=True, ascending=True,bins=None,dropna=False).head(30)

In [48]:
df['Genre'].value_counts(normalize=False, sort=True, ascending=True,bins=None,dropna=False)

NaN               50
Puzzle           578
Strategy         680
Fighting         847
Simulation       865
Platform         884
Racing          1243
Adventure       1280
Shooter         1308
Role-Playing    1483
Misc            1734
Sports          2341
Action          3305
Name: Genre, dtype: int64

In [58]:
# 데이터 결측치 확인
df.isnull().sum()

Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

In [69]:
# 결측치 있는 행 어떻게 생겼는지 확인
cond = df['Year'].isnull() == True
cond1 = df['Genre'].isnull() == True
cond2 = df['Publisher'].isnull() == True
df[cond]

,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Name,,,,,,,,
Rock Band,X360,NaN,Misc,Electronic Arts,1.93,0.34,0,0.21
Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales),Wii,NaN,Racing,Unknown,0,0,0.02,0
Jewel Link Chronicles: Mountains of Madness,DS,NaN,Puzzle,Avanquest,0,0.06,0,0.01
Majesty 2: The Fantasy Kingdom Sim,X360,NaN,Simulation,Unknown,0.03,0,0,0
Famista 64,N64,NaN,Sports,Namco Bandai Games,0,0,0.17,0.03
...,...,...,...,...,...,...,...,...
Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0,0,0
Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0M,0.03
Madden NFL 11,Wii,NaN,Sports,Unknown,0.7,0,0,50K


### **1-2. 데이터 확인 후 전처리 계획**

1. 데이터 타입 관련

  1) NA_Sales,	EU_Sales,	JP_Sales,	Other_Sales (각 나라별 출고량)

  ```
  (1) 현재 object 타입으로 =>  int 타입 변환 필요
  (2) 소수와 k,m 등 단위 통일 필요 (k = 천 , m = 백만)
  ```

  2) Year (출시 연도)
  ```
  (1) N/A , 2015(4자리 연도), 15(2자리 연도), 0(2000년?)(한자리 연도) 섞여있음 
      => 결측치 처리, 4자리로 통일 필요
  ```
  3) 알맞은 feature에 안들어가고 한칸씩 밀린 데이터 (사진 예시 : 엑셀 데이터 캡쳐) 
    image.png
    image.png
      image.png
    ```
  그러나! 엑셀과 달리 파이썬에서는 제대로 출력될 수 도 있으니, 둘 다 확인봐야한다!!! **
  ```



2. 결측치 관련 (isnull().sum() != 0)

  1) Year, Genre, Publisher

  ```
  (1) Year : 고민 필요 >> 해당 행 지우기? ( 공통 사항 : 연도별 게임 트랜드)
  (2) Genre : 고민 필요 > 해당 행 지우기? ( 공통 사항 : 지역별 선호 장르)
  (3) Publisher : 상관 X?
  ```

In [ ]:
# sales feature 숫자 단위 통일 : M(백만) 단위로 통일

#1. M 없애기
#2. K를 기준으로 그 앞 문자(숫자의 모습을 한 문자)를 int 타입으로 만들기 
#3. int 타입으로 만들어진 것에 1000(k)를 곱하고, 거기에 1000000(m)을 나눠줘서 백만 단위로 만들어주기 (k = 1000, m = 1000000)



In [75]:
na_sales = []
for i in df['NA_Sales']:
  na_sales.append(i)

print(na_sales)

['0.04', '0.17', '0', '0.04', '0.12', '0', '0.02', '0.07', '0', '0.04', '480K', '0.14', '0', '0.02', '0', '0.14', '0.01', '0', '0.02', '0', '0.57', '0.06', '0.08', '0', '0.48', '0.02', '0.03', '0', '0', '0.14', '0.05', '1.93', '0.12', '0', '0.29', '0.19', '0.3', '0.26', '0.11', '0.18', '0.3', '0', '0.33', '0', '0.51', '60K', '0', '0.06', '0.13', '0.14', '0.52', '0.18', '0', '0.34', '0.05', '0.25', '0', '0', '0.41', '0.18', '0.02', '0.25', '0.13', '0.08', '0.01', '0', '0.52', '0.04', '0.08', '0.18', '0.22', '0.03', '0.08', '0.22', '0.01', '0.08', '0', '0.35', '0.1', '0', '0.08', '0.1', '0', '0.19', '0.13', '0.17', '0', '0.08', '0.13', '0.22', '0.07', '0.17', '0.67', '0.05', '0', '0.07', '0.02', '0.59', '0.36', '0', '0', '0.18', '0', '0.05', '0.21', '0.06', '0.19', '1.19', '0', '0', '0', '0.01', '0', '0.04', '0.04', '0', '0.04', '0', '0.03', '0', '0', '0.07', '5.91', '0.04', '0', '0', '1.98', '0.04', '0', '0.02', '0.13', '0', '0', '0.02', '0.35', '0.07', '0', '0.12', '0.04', '0.18', '0',

In [138]:
#1. M 없애기
def remove_M(df):
  no_m_list = []
  for i in df:
    x = i.replace('M','')
    no_m_list.append(x)
  df = no_m_list
  return df

df['NA_Sales'] = remove_M(df['NA_Sales'])
df['EU_Sales'] = remove_M(df['EU_Sales'])
df['JP_Sales'] = remove_M(df['JP_Sales'])
df['Other_Sales'] = remove_M(df['Other_Sales'])

print(df[['NA_Sales','EU_Sales']].head(20))
print(df[['JP_Sales','Other_Sales']].tail(20))

                                                   NA_Sales EU_Sales
Name                                                                
Candace Kane's Candy Factory                           0.04        0
The Munchables                                         0.17        0
Otome wa Oanesama Boku ni Koi Shiteru Portable            0        0
Deal or No Deal: Special Edition                       0.04        0
Ben 10 Ultimate Alien: Cosmic Destruction              0.12     0.09
Power Pro Kun Pocket 8                                    0        0
JASF: Jane's Advanced Strike Fighters                  0.02     0.01
Samurai Warriors: State of War                         0.07        0
Imabikisou                                                0        0
The Penguins of Madagascar: Dr. Blowhole Return...     0.04     0.03
Ford Racing                                            480K     0.33
Harvest Moon: More Friends of Mineral Town             0.14     0.05
Mount & Blade                     

In [ ]:
## K => M단위로 처리하기 (K 없애기 > 수치화 > *1000 > 나누기 1000000)

'''
K를 다 없앤 후에는 뭐가 K가 붙어 있던 애인지 모름 >> K있는 애들만 뽑아서 그래서 한번에 해줘야함. 
'''

#얘 보고 다시 생각하기 
''' 
for i in df['NA_Sales'].values : #해당 feature의 값들만 가지고 뽑아낼 수 있음 (column 통으로 가져왔을 때 안됐고(series), 리스트에 넣어서 가져와도 안됐음)
  if 'K' in i:
    for i in df['NA_Sales'].values:
      x = i.replace('K','')
      x_num = float(x)
      x_to_M = (x_num*1000)/1000000
  else:
    pass
'''

def remove_K(df):
  no_k_list = []
  for i in df:
    x = i.replace('K','')
    no_k_list.append(x)
  df = no_k_list
  return df

  
df['NA_Sales'] = remove_K(df['NA_Sales'])
df['EU_Sales'] = remove_K(df['EU_Sales'])
df['JP_Sales'] = remove_K(df['JP_Sales'])
df['Other_Sales'] = remove_K(df['Other_Sales'])

print(df[['NA_Sales','EU_Sales']].head(20))
print(df[['JP_Sales','Other_Sales']].head(20))

In [ ]:
# x_to_M = (x_num*1000)/1000000

In [ ]:
for i in df['NA_Sales'].values:
  x = i.replace('K','')
    x_num = float(x)
    x_to_M = (x_num*1000)/1000000
  else:
    pass
df['NA_Sales'] = x_to_M
print(df['NA_Sales'].head(20))

In [92]:
na_no_K=[]
for i in na_sales:
  x1 = i.split('K')[0]
  na_no_K.append(x1)
na_no_K[:100]

['0.04',
 '0.17',
 '0',
 '0.04',
 '0.12',
 '0',
 '0.02',
 '0.07',
 '0',
 '0.04',
 '480',
 '0.14',
 '0',
 '0.02',
 '0',
 '0.14',
 '0.01',
 '0',
 '0.02',
 '0',
 '0.57',
 '0.06',
 '0.08',
 '0',
 '0.48',
 '0.02',
 '0.03',
 '0',
 '0',
 '0.14',
 '0.05',
 '1.93',
 '0.12',
 '0',
 '0.29',
 '0.19',
 '0.3',
 '0.26',
 '0.11',
 '0.18',
 '0.3',
 '0',
 '0.33',
 '0',
 '0.51',
 '60',
 '0',
 '0.06',
 '0.13',
 '0.14',
 '0.52',
 '0.18',
 '0',
 '0.34',
 '0.05',
 '0.25',
 '0',
 '0',
 '0.41',
 '0.18',
 '0.02',
 '0.25',
 '0.13',
 '0.08',
 '0.01',
 '0',
 '0.52',
 '0.04',
 '0.08',
 '0.18',
 '0.22',
 '0.03',
 '0.08',
 '0.22',
 '0.01',
 '0.08',
 '0',
 '0.35',
 '0.1',
 '0',
 '0.08',
 '0.1',
 '0',
 '0.19',
 '0.13',
 '0.17',
 '0',
 '0.08',
 '0.13',
 '0.22',
 '0.07',
 '0.17',
 '0.67',
 '0.05',
 '0',
 '0.07',
 '0.02',
 '0.59',
 '0.36',
 '0']